# First, explore possabilities with OpenAI and Agent based LLM

## Installs and imports

In [ ]:
!pip install langchain -q

In [ ]:
!pip install openai -q

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ''

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain import HuggingFaceHub, SQLDatabaseChain, OpenAI
from langchain import PromptTemplate, LLMChain

In [ ]:
!pip install duckdb==0.8.0

In [ ]:
import duckdb
duckdb.__version__

## 🦆 DuckDB Setup

In [ ]:
import duckdb

In [ ]:
!pip install duckdb-engine -q

In [ ]:
from sqlalchemy import Column, Integer, Sequence, String, create_engine

uri = 'duckdb:///../ddb/duckdb-course/nba.duckdb'
    
connect_args = {
        'read_only': True
    }

CONN = create_engine(uri)


In [ ]:
con = duckdb.connect('../ddb/duckdb-course/nba.duckdb')

In [ ]:
con.query('describe nba;')

In [ ]:
con.query('from nba limit 1').df()

# Create a modified table if needed

In [ ]:
con.query('''create table nba_short as select * exclude(column00, "Unnamed: 0.1", "Unnamed: 0") from nba    ;''')

In [ ]:
con.query('alter table nba_short add column match_date date')

In [ ]:
con.query('''update nba_short set match_date = strptime(left(match_id, 8), '%Y%m%d')''')

# Set up LangChain 

In [ ]:
db = SQLDatabase.from_uri(
    uri,
    include_tables=['nba_short'], 
	sample_rows_in_table_info=3)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
template = """/
You are a SQL Analyst that is querying a database of NBA shots that record data about every shot taken in the NBA.

Below is a description of the columns, data types, and information in the columns:

The column name match_id with the data type VARCHAR contains the following information: Unique identifier for every match or game
The column name shots with the data type DOUBLE PRECISION contains the following information: Location on the court on the X axis
The column name shotY with the data type DOUBLE PRECISION contains the following information: Location on the court on the Y axis
The column name quarter with the data type VARCHAR contains the following information: Quarter of the game out of 4 when the shot was taken
The column name time_remaining with the data type VARCHAR contains the following information: Amount of time remaining in the game in which the shot was taken
The column name player with the data type VARCHAR contains the following information: Full name of the player taking the shot
The column name team with the data type VARCHAR contains the following information: Team name abbreviation of the team of the player taking the shot
The column name made with the data type BOOLEAN contains the following information: Boolean that represents if the shot was made or not
The column name shot_type with the data type VARCHAR contains the following information: Description of the type of shot taken
The column name distance with the data type VARCHAR contains the following information: Distance in feet of the shot from the basket (0 is a dunk)
The column name score with the data type VARCHAR contains the following information: Score of the game after the shot was taken
The column name opp with the data type VARCHAR contains the following information: Opponent team of the player taking the shot
The column name status with the data type VARCHAR contains the following information: Text description of the score status of the game
The column name match_date with the data type VARCHAR contains the following information: Date the match or game took place

Your job is to write an execute a query that answers the following question:
{query}
"""

prompt = PromptTemplate.from_template(template)

agent_executor.run(
    prompt.format(query = "Who scored the most 3-pointers in 2015 with at least 100 shots attempted?")
)

# Let's try it with spatial data

In [ ]:
con.query(''' install spatial ''')

In [ ]:
con.query(''' load spatial ''')

In [ ]:
con.query(''' create or replace table nyc_311 as select * exclude('Incident Zip') from '../ddb/duckdb-course/spatial/nyc_311.csv';
 ''')

In [ ]:
con.query('alter table nyc_311 add column geom geometry;')

In [ ]:
con.query('update nyc_311 set geom = st_point(Longitude, Latitude);')

In [ ]:
con.query('''
create or replace table nyc_nh as
select *, st_geomfromwkb(wkb_geometry) as geom from st_read('../ddb/duckdb-course/spatial/nyc_hoods.geojson');
''')

In [ ]:
con.query('''
create or replace table nyc_311_s as 
select "Created Date",
"Agency Name",
"Complaint Type", 
"Descriptor", 
"Incident Address",
geom
from nyc_311
''')

In [ ]:
sdb = SQLDatabase.from_uri(
    uri,
    include_tables=['nyc_311_s', 'nyc_nh'], 
	sample_rows_in_table_info=3)

In [ ]:
toolkit = SQLDatabaseToolkit(db=sdb, llm=OpenAI(temperature=0))

In [ ]:
agent_executor_spatial = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
template = """/
You are a SQL Analyst that is querying a database that contains two tables: One of 311 calls in New York City and New York Neighborhoods.

Both columns have a geometry which can be used to perform spatial joins using spatial functions commonly found in PostGIS.

Your job is to write an execute a query that answers the following question:
{query}
"""

prompt = PromptTemplate.from_template(template)

agent_executor_spatial.run(
    prompt.format(query = "How many 311 calls about graffiti took place in the West Village?")
)